# Building Your First Multimodal GenAI App

![Alt text](img/multimodal_app_1.png)

In this notebook, we'll explore a variety of SOTA GenAI models and get a sense of how to stitch them together!

## Get our API Keys in our environment

- Create a [Groq](https://groq.com/) account and navigate [here to get your API key](https://console.groq.com/keys). They have a free tier with a bunch of LLMs (see screenshot below)!
- If you'd prefer to use OpenAI, you can do that and get [your API key here](https://platform.openai.com/api-keys).
- To use the models below as is, you'll need a [Replicate account](https://replicate.com/). If you're using this notebook in a workshop, chances are Hugo is able to provision free Replicate credits for you so ask him, if he hasn't mentioned it.
- Many of these models [you can also find on HuggingFace](https://huggingface.co/models), if you'd prefer.

![Alt text](img/multimodal_app_2.png)

In [1]:
import getpass


# Prompt for the Replicate API key
replicate_api_key = getpass.getpass("Please enter your Replicate API key: ")
print("Replicate API key captured successfully!")

# Prompt for the Grok API key
groq_api_key = getpass.getpass("Please enter your Groq API key: ")
print("Groq API key captured successfully!")

# # Prompt for the OpenAI API key
# openai_api_key = getpass.getpass("Please enter your OpenAI API key: ")
# print("Replicate OpenAI key captured successfully!")


Replicate API key captured successfully!
Groq API key captured successfully!


## Suno Bark: text to audio

First up, we'll experiment with the [Suno Bark](https://github.com/suno-ai/bark) text to audio model:

In [2]:
import replicate

# Create a Replicate client instance with the API token
client = replicate.Client(api_token=replicate_api_key)

# Define the input parameters for the model
input_params = {
    "prompt": "Hello, my name is Hugo. And, uh — and I like pizza. [laughs] But I also have other interests such as playing chess. [chuckles]",
    "text_temp": 0.7,
    "output_full": False,
    "waveform_temp": 0.7,
    "history_prompt": "announcer"
}

# Run the model using Replicate API
try:
    output = client.run(
        "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
        input=input_params
    )
    print(output)
except Exception as e:
    print(f"Error: {e}")

{'audio_out': 'https://replicate.delivery/czjl/qkXoJrrhbJIXEdblDtEJFjw2AoTefj4UN2tZLWVeVdLYBpjmA/audio.wav'}


### LLM output --> Suno bark

But what if we want to pipe the output of an LLM into Bark?

In [3]:
from groq import Groq

def get_llm_response(user_input):
    client = Groq(
        api_key=groq_api_key)

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model="llama3-8b-8192",
    )

    return response.choices[0].message.content

# from openai import OpenAI
# import os


# def get_llm_response(user_input):
#     client = OpenAI(api_key=openai_api_key)
    
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo-0613",
#         messages=[
#     {"role": "user", "content": user_input}
#   ]
#         )
#     return response.choices[0].message.content

In [4]:
song = get_llm_response("a short pirates sea shanty")
print(song)

(Verse 1)
Oh, we set sail on the "Black Swan's wake"
With a crew o' scurvy dogs and a heart o' ache
We've got a map and a cunning guile
To find the treasure, we'll sail for a while

(Chorus)
Heave ho, me hearties, let the anchors go
Heave ho, me hearties, the wind's in tow
Heave ho, me hearties, the sun's on our back
Heave ho, me hearties, the treasure we'll attack

(Verse 2)
The cogs are creakin', the chains are tight
We're bound for the island, where the gold will shine bright
With cutlasses sharp and a wink in our eye
We'll plunder the riches, or die tryin' to get by

(Chorus)
Heave ho, me hearties, let the anchors go
Heave ho, me hearties, the wind's in tow
Heave ho, me hearties, the sun's on our back
Heave ho, me hearties, the treasure we'll attack

(Bridge)
Yarrr, we've battled storms and fought our share
Of battles with the Kraken, and the sea dogs' glare
But we'll never give in, nor ever give up the quest
For the treasure's callin', and we must pass the test

(Chorus)
Heave ho,

In [5]:
# Define the input parameters for the model
input_params = {
    "prompt": song,
    "text_temp": 0.7,
    "output_full": False,
    "waveform_temp": 0.7,
    "history_prompt": "announcer",
   # "duration": 30
}

# Run the model using Replicate API
try:
    output = client.run(
        "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
        input=input_params
    )
    print(output)
except Exception as e:
    print(f"Error: {e}")

{'audio_out': 'https://replicate.delivery/czjl/utMjOV1tA46DBJRcfapVYCIAYqrTjsCjEwABYVfrFVg0h0RTA/audio.wav'}


This is totally bent and makes no sense ☝️

## Text to music w/ meta musicgen

What if we wanted to create some music with text? Let's try Musicgen from Meta.

In [6]:
input = {
    "prompt": "Horns and Drums. Edo25 major g melodies that sound triumphant and cinematic. Leading up to a crescendo that resolves in a 9th harmonic",
    "model_version": "stereo-large",
    "output_format": "mp3",
    "normalization_strategy": "peak"
}

output = client.run(
    "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
    input=input
)
print(output)
#=> "https://replicate.delivery/pbxt/OeLYIQiltdzMaCex1shlEFy6...

https://replicate.delivery/yhqm/VwBSroelZ40WTiegPajrzCScf0CHT9K6yCZKMGmIeH0yLSHNB/out.mp3


In [7]:
input = {
    "prompt": "Ancient Trip Hop with Throat Singing",
    "model_version": "stereo-large",
    "output_format": "mp3",
    "normalization_strategy": "peak",
    "duration": 30 
}

output = client.run(
    "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
    input=input
)
print(output)
#=> "https://replicate.delivery/pbxt/OeLYIQiltdzMaCex1shlEFy6...

https://replicate.delivery/yhqm/tn92XRiUQUZmEVe3Gb7B2ffKzEafknfyhh2o8zwb8o9NikOaC/out.mp3


## Text to music with riffusion

There are lots of other models to experiment with, such as riffusion:

In [8]:
output = client.run(
    "riffusion/riffusion:8cf61ea6c56afd61d8f5b9ffd14d7c216c0a93844ce2d82ac1c9ecc9c7f24e05",
    input={
        "alpha": 0.5,
        "prompt_a": "West African Desert Blues",
        "prompt_b": "Throat Singing",
        "denoising": 0.75,
        "seed_image_id": "vibes",
        "num_inference_steps": 50
    }
)
print(output)

{'audio': 'https://replicate.delivery/czjl/0USXXIEfI60cDi1aUee4PvswzzexTgPe5AdfjE1ZCyFmGJd0E/gen_sound.wav', 'spectrogram': 'https://replicate.delivery/czjl/Uor64JTeqh1UWikhhofJmTh3G2nCXPQXKbCrcCf2Bqp2IpjmA/spectrogram.jpg'}


___

## Experiment: One prompt to many models

Now what if we wanted to use a single prompt to create text, audio, images, and video?

In [9]:
message = "The Waffle House is really messing up the pancakes and bacon tonight HOLY MOLEY and there's anarchist jazz also!"

### text to image

In [10]:
input = {
    "prompt": message
}

output = client.run(
    "fofr/epicrealismxl-lightning-hades:0ca10b1fd361c1c5568720736411eaa89d9684415eb61fd36875b4d3c20f605a",
    input=input
)
print(output)
#=> ["https://replicate.delivery/pbxt/ulYZRIyAUDYpOZfl7OjhrKx...

['https://replicate.delivery/pbxt/zWNJ3cURdgb3MVLEqSO2veRfvsoV86SNnSqueel6M5eM3kOaC/R8__00001_.webp']


### text to audio

In [11]:
# Define the input parameters for the model
input_params = {
    "prompt": message,
    "text_temp": 0.7,
    "output_full": False,
    "waveform_temp": 0.7,
    "history_prompt": "announcer",
   # "duration": 30
}

# Run the model using Replicate API
try:
    output = client.run(
        "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
        input=input_params
    )
    print(output)
except Exception as e:
    print(f"Error: {e}")

{'audio_out': 'https://replicate.delivery/czjl/6mm9Of8I6jUbKSfJRu72mbpDaWg1Ikh54Gex5e2AeU2f0Jd0E/audio.wav'}


### text to music

In [12]:
input = {
    "prompt": message,
    "model_version": "stereo-large",
    "output_format": "mp3",
    "normalization_strategy": "peak",
    "duration": 30 
}

output = client.run(
    "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
    input=input
)
print(output)
#=> "https://replicate.delivery/pbxt/OeLYIQiltdzMaCex1shlEFy6...

https://replicate.delivery/yhqm/d7H1tBpfjfoPbkspgqw3fDfeeQuX6II5prineWU1HqcxMU6oJA/out.mp3


## Many models at once

Let's write some utility functions that use these models:

In [13]:
def generate_epic_realism(prompt, api_token):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_data = {
        "prompt": prompt
    }

    # Run the model using Replicate API
    output = client.run(
        "fofr/epicrealismxl-lightning-hades:0ca10b1fd361c1c5568720736411eaa89d9684415eb61fd36875b4d3c20f605a",
        input=input_data
    )
    
    return output



def generate_suno_bark(prompt, api_token, text_temp=0.7, output_full=False, waveform_temp=0.7, history_prompt="announcer"):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_params = {
        "prompt": prompt,
        "text_temp": text_temp,
        "output_full": output_full,
        "waveform_temp": waveform_temp,
        "history_prompt": "zh_speaker_7",
    }

    # Run the model using Replicate API
    try:
        output = client.run(
            "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
            input=input_params
        )
        return output
    except Exception as e:
        print(f"Error: {e}")
        return None




def generate_music_gen(prompt, api_token, duration=30, model_version="stereo-large", output_format="mp3", normalization_strategy="peak"):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_data = {
        "prompt": prompt,
        "model_version": model_version,
        "output_format": output_format,
        "normalization_strategy": normalization_strategy,
        "duration": duration 
    }

    # Run the model using Replicate API
    output = client.run(
        "meta/musicgen:671ac645ce5e552cc63a54a2bbff63fcf798043055d2dac5fc9e36a837eedcfb",
        input=input_data
    )
    
    return output


def generate_suno_bark(prompt, api_token, text_temp=0.7, output_full=False, waveform_temp=0.7, history_prompt="announcer"):
    # Create a Replicate client instance with the API token
    client = replicate.Client(api_token=replicate_api_key)

    # Define the input parameters for the model
    input_params = {
        "prompt": prompt,
        "text_temp": text_temp,
        "output_full": output_full,
        "waveform_temp": waveform_temp,
        "history_prompt": "announcer",
    }

    # Run the model using Replicate API
    try:
        output = client.run(
            "suno-ai/bark:b76242b40d67c76ab6742e987628a2a9ac019e11d56ab96c4e91ce03b79b2787",
            input=input_params
        )
        return output
    except Exception as e:
        print(f"Error: {e}")
        return None






Let's test them out:

In [14]:
message = "crazy wild zombie party at the blaring symphony orchestra"
output = generate_epic_realism(message, replicate_api_key)
print(output)

output = generate_suno_bark(message, replicate_api_key)
print(output)

output = generate_music_gen(message, replicate_api_key)
print(output)


['https://replicate.delivery/pbxt/tsOFefkbVjtQt0fBwMAN2va8vKTynZgCToAYCeHqdhz3nSHNB/R8__00001_.webp']
{'audio_out': 'https://replicate.delivery/czjl/Tz52jg7f86yyPCwfdbogwRhnfExD4BNFFlNpMsEGMyYeoSHNB/audio.wav'}
https://replicate.delivery/yhqm/igcvsNxfwT2kXCTtIaQDmnERuXg6a4q3Qw7TyQWqWtNqV6oJA/out.mp3


In [15]:
# Define your API token and prompt message
# api_token = 'your_api_token_here'
message = "The Waffle House messing it up for real with the pancakes and bacon and punk abstract jazz, yo!"

# Run the Epic Realism model
epicrealism_output = generate_epic_realism(message, replicate_api_key)
print("Epic Realism Output:")
print(epicrealism_output)

# Run the Meta MusicGen model
musicgen_output = generate_music_gen(message, replicate_api_key)
print("Meta MusicGen Output:")
print(musicgen_output)

# Run the Suno Bark model
bark_output = generate_suno_bark(message, replicate_api_key)
print("Suno Bark Output:")
print(bark_output)



Epic Realism Output:
['https://replicate.delivery/pbxt/bDyvJtzNTTL0BZ7RDf4iwtAnMBcld1oGBjDDPGAswHJsV6oJA/R8__00001_.webp']
Meta MusicGen Output:
https://replicate.delivery/yhqm/0ViYMJoNoe0xXCOscAnB6dwZR8uJou2MCKHOqJZ6flzeYpjmA/out.mp3
Suno Bark Output:
{'audio_out': 'https://replicate.delivery/czjl/FHxwoMFdGV6vNBbpakk5UWaaq2oeNDvmfXqjCCDQ29S2s0RTA/audio.wav'}


### Experiment: text to video

In [16]:
message = "The Waffle House messing it up for real with the pancakes and bacon and punk abstract jazz, yo!"

input = {
    "sampler": "klms",
    "max_frames": 100,
    "animation_prompts": message
}

output = client.run(
    "deforum/deforum_stable_diffusion:e22e77495f2fb83c34d5fae2ad8ab63c0a87b6b573b6208e1535b23b89ea66d6",
    input=input
)
print(output)
#=> "https://replicate.delivery/mgxm/873a1cc7-0427-4e8d-ab3c-...

https://replicate.delivery/yhqm/u6chBKBnYZqAKZhI0CpnyGHGgJzqfPeZW07AmUCUlWSqv0RTA/out.mp4
